In [1]:
%run ../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb
import os

import mne
from mne.preprocessing import ICA
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (50.0, 30.0)
matplotlib.rcParams.update({'font.size': 15})

from mne.datasets import sample
from mne.decoding import UnsupervisedSpatialFilter

from sklearn.decomposition import PCA
import pandas as pd

In [2]:
path = 'E:\EpochedEEG'
epoch_files = os.listdir(path)

In [3]:
file = os.path.join(path,epoch_files[0])

In [4]:
epoch = mne.read_epochs(file, preload = True)

Reading E:\EpochedEEG\20131216_1441_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
5424 matching events found
Applying baseline correction (mode: mean)
5424 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [5]:
pca = UnsupervisedSpatialFilter(PCA(9), average=False) # PCA, keep 9 components 

epoch_data = epoch.get_data()
pca_data = pca.fit_transform(epoch_data)

In [6]:
gamma_epoch = epoch.filter(l_freq=30, h_freq= 100)
beta_epoch =  epoch.filter(l_freq=12, h_freq= 30)
alpha_epoch = epoch.filter(l_freq=8, h_freq= 12)
theta_epoch = epoch.filter(l_freq=4, h_freq= 7)
delta_epoch = epoch.filter(l_freq=0, h_freq= 4)

Setting up band-pass filter from 30 - 1e+02 Hz
l_trans_bandwidth chosen to be 7.5 Hz
h_trans_bandwidth chosen to be 25.0 Hz
Filter length of 113 samples (0.441 sec) selected
Setting up band-pass filter from 12 - 30 Hz
l_trans_bandwidth chosen to be 3.0 Hz
h_trans_bandwidth chosen to be 7.5 Hz
Filter length of 283 samples (1.105 sec) selected
filter_length (283) is longer than the signal (180), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-6-d74bec87c4a9>:2: RuntimeWarning: filter_length (283) is longer than the signal (180), distortion is likely. Reduce filter length or filter a longer signal.
  beta_epoch =  epoch.filter(l_freq=12, h_freq= 30)


Setting up band-pass filter from 8 - 12 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 3.0 Hz
Filter length of 423 samples (1.652 sec) selected
filter_length (423) is longer than the signal (180), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-6-d74bec87c4a9>:3: RuntimeWarning: filter_length (423) is longer than the signal (180), distortion is likely. Reduce filter length or filter a longer signal.
  alpha_epoch = epoch.filter(l_freq=8, h_freq= 12)


Setting up band-pass filter from 4 - 7 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 2.0 Hz
Filter length of 423 samples (1.652 sec) selected
filter_length (423) is longer than the signal (180), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-6-d74bec87c4a9>:4: RuntimeWarning: filter_length (423) is longer than the signal (180), distortion is likely. Reduce filter length or filter a longer signal.
  theta_epoch = epoch.filter(l_freq=4, h_freq= 7)


Setting up low-pass filter at 4 Hz
h_trans_bandwidth chosen to be 2.0 Hz
Filter length of 423 samples (1.652 sec) selected
filter_length (423) is longer than the signal (180), distortion is likely. Reduce filter length or filter a longer signal.


<ipython-input-6-d74bec87c4a9>:5: RuntimeWarning: filter_length (423) is longer than the signal (180), distortion is likely. Reduce filter length or filter a longer signal.
  delta_epoch = epoch.filter(l_freq=0, h_freq= 4)


In [7]:
gamma_data = gamma_epoch.get_data()
beta_data = beta_epoch.get_data()
alpha_data = alpha_epoch.get_data()
theta_data = theta_epoch.get_data()
delta_epoch = delta_epoch.get_data()

In [8]:
gamma_pca_data = pca.fit_transform(gamma_data)
beta_pca_data = pca.fit_transform(beta_data)
alpha_pca_data = pca.fit_transform(alpha_data)
theta_pca_data = pca.fit_transform(theta_data)
delta_pca_data = pca.fit_transform(delta_epoch)# didn't name this correctly

## average data from each band

In [9]:
gamma_pca_data.shape

(5424, 9, 180)

In [10]:
# final_data = np.shape()
final_means = []
for data in gamma_pca_data:
#     print (data.shape)
    means = []
    for channel in data:
#         print(channel.shape)
        mean = channel.mean()
        means.append(mean)
    final_means.append(means)
final_means = np.array(final_means)

In [11]:
final_means.shape

(5424, 9)

In [12]:
def get_mean_band(data):
    final_means = []
    for data in gamma_pca_data:
    #     print (data.shape)
        means = []
        for channel in data:
    #         print(channel.shape)
            mean = channel.mean()
            means.append(mean)
        final_means.append(means)
    
    return np.array(final_means)
    

In [13]:
gamma_final_mean = get_mean_band(gamma_pca_data)
beta_final_mean = get_mean_band(beta_pca_data)
alpha_final_mean = get_mean_band(alpha_pca_data)
delta_final_mean = get_mean_band(delta_pca_data)
theta_final_mean = get_mean_band(theta_pca_data)
regular_data_final_mean = get_mean_band(pca_data)                                 

In [14]:
results = np.shape((len(epoch.events), 9, 63))

In [56]:
results = np.concatenate((gamma_final_mean,
                          beta_final_mean,
                          alpha_final_mean,
                          delta_final_mean,
                          theta_final_mean,
                          regular_data_final_mean),axis = 1)

In [57]:
results.shape

(5424, 54)

## extract congruent & incongruent labels

In [16]:
df = pd.DataFrame()

In [17]:
results.shape

(5424, 54)

In [28]:
gamma_final_mean.shape

(5424, 9)

In [21]:
concat_array.shape

(5424, 9, 63)

In [24]:
concat_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [40]:
gamma_final_mean_3d = gamma_final_mean[:,np.newaxis,:]
gamma_final_mean_3d.shape

(5424, 1, 9)

In [51]:
gamma_final_mean_3d = gamma_final_mean[...,np.newaxis]
beta_final_mean_3d = beta_final_mean[...,np.newaxis]
alpha_final_mean_3d = alpha_final_mean[...,np.newaxis]
delta_final_mean_3d = delta_final_mean[...,np.newaxis]
theta_final_mean_3d = theta_final_mean[...,np.newaxis]
regular_final_mean_3d = regular_data_final_mean[...,np.newaxis]

In [54]:
results = np.concatenate((gamma_final_mean_3d,
                          beta_final_mean_3d,
                          alpha_final_mean_3d,
                          delta_final_mean_3d,
                          theta_final_mean_3d,
                          regular_final_mean_3d), axis = 2)

In [55]:
results.shape

(5424, 9, 6)